In [1]:
import pdfplumber
import transformers
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch
from transformers import BitsAndBytesConfig

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
path = 'WWF_Plastic_Policy_Summit_2024_Key_Takeaways_.pdf'

with pdfplumber.open(path) as pdf: 
    content = ''
    for i in range(len(pdf.pages)):
        # 讀取PDF文件的第i+1頁
        page = pdf.pages[i] 
 
        # 讀取文本內容，去除頁碼
        page_content = '\n'.join(page.extract_text().split('\n')[:-1])
        content = content + page_content
 
print(content)

c:\Users\jywun\anaconda3\envs\module\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda
Key TakeawaysTable of Contents
Introduction ............................................................................................................................ 2
Introduction
Overarching Themes ............................................................................................................. 4
Driving Reduction and Reuse through Legislation ........................................................... 6
Decreasing Harmful Impacts of Plastics ............................................................................. 7
Reaching the Public: Circularity Education & Outreach ................................................... 9
Connecting the Dots: State Policy ..................................................................................... 10
Optimizing Product Design ................................................................................................. 11
Leveraging Finance for Change ...............................................................

In [2]:
text_splitter_question_gen = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
text_chunks_question_gen = text_splitter_question_gen.split_text(content)
docs_question_gen = [Document(page_content=t) for t in text_chunks_question_gen]
docs_question_gen

[Document(page_content='源大環能股份有限公司\n誠信經營守則\n一、目的\n為建立誠信經營之企業文化及健全發展，提供其建立良好商業運作之參考架構，特訂定\n本守則。\n二、範圍\n本公司、子公司及其他具有實質控制能力之機構與組織。\n三、權責\n董事長室：負責定期檢視內部獨立檢舉信箱及保護檢舉人員。\n四、名詞定義\n本守則所稱利益，其利益係指任何有價值之事物，包括任何形式或名義之金錢、餽贈、\n佣金、職位、服務、優待、回扣等。但屬正常社交禮俗，且係偶發而無影響特定權利義\n務之虞時，不在此限。\n五、作業說明\n1.禁止不誠信行為\n本公司之董事、經理人、受僱人、受任人或具有實質控制能力者(以下簡稱本公司人員)，\n於從事商業行為之過程中，不得直接或間接提供、承諾、要求或收受任何不正當利益，\n或做出其他違反誠信、不法或違背受託義務等不誠信行為，以求獲得或維持利益。(以\n下簡稱不誠信行為)\n前項行為之對象，包括公職人員、參政候選人、政黨或黨職人員，以及任何公、民營\n企業或機構及其董事（理事）、監察人（監事）、經理人、受僱人、實質控制者或其他\n利害關係人。\n2.法令遵循'),
 Document(page_content='前項行為之對象，包括公職人員、參政候選人、政黨或黨職人員，以及任何公、民營\n企業或機構及其董事（理事）、監察人（監事）、經理人、受僱人、實質控制者或其他\n利害關係人。\n2.法令遵循\n公司應遵守公司法、證券交易法、商業會計法、政治獻金法、貪污治罪條例、政府採\n購法、公職人員利益衝突迴避法、個人資料保護法、公開發行公司相關規章或其他商\n業行為有關法令，以作為落實誠信經營之基本前提。\n3.公司政策\n本公司於廉潔、透明及負責之經營理念，制定以誠信為基礎之政策，經董事會通過，\n並建立良好之公司治理與風險控管機制，以創造永續發展之經營環境。\n4.防範措施\n本公司應建立不誠信行為風險之評估機制，定期分析及評估營業範圍內具較高不誠信\n行為風險之營業活動，據以訂定防範方案並定期檢討防範方案之妥適性與有效性。\n本公司人員嚴禁以下之行為:\n4.1.行賄及收賄\n本公司人員於執行業務時，不得直接或間接提供、承諾、要求或收受任何形式之\n不正當利益，包括回扣、佣金、疏通費或透過其他途徑向客戶、代理商、承包商、法

In [3]:
prompt_template_questions = """
    你是一個根據提供內容產生中文問題的專家。
    你的目標是幫助使用者了解文件內容。
    你可以通過詢問和以下內容有關的問題來做到這一點：

    ------------
    {text}
    ------------

    產生中文問題，確保不要遺失任何重要信息。

    QUESTIONS:
    """

PROMPT_QUESTIONS = PromptTemplate(template=prompt_template_questions, input_variables=["text"])

refine_template_questions = """
    你是一個根據提供內容產生中文問題的專家。
    你的目標是幫助一個使用者了解文件內容。我們已經收到一些生成出來的問題，但還不夠全面: {existing_answer}.
    我們可以選擇完善現有的問題或添加新的問題。

    ------------
    {text}
    ------------

    根據提供的上下文產生中文問題，如果提供的上下文沒有幫助，保持原始問題。          
    QUESTIONS:
    """
REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template_questions,
)

In [4]:
model_checkpoint = "C:/Users/jywun/Desktop/NYCU/模組/QA_ARAG/llama-3-8B"
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, torch_dtype=torch.float16).cuda()
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
pipeline = pipeline(
        "text-generation",
        model=model_checkpoint,
        tokenizer=tokenizer,
        max_new_tokens=512,
        device=0,
    )
llm = HuggingFacePipeline(pipeline=pipeline) 

Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.73s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:10<00:00,  2.63s/it]
c:\Users\jywun\anaconda3\envs\module\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  warn_deprecated(


In [5]:
question_gen_chain = load_summarize_chain(llm=llm, question_prompt=PROMPT_QUESTIONS,refine_prompt=REFINE_PROMPT_QUESTIONS, verbose=True, chain_type="refine")

In [6]:
questions = question_gen_chain.run(docs_question_gen)

c:\Users\jywun\anaconda3\envs\module\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    你是一個根據提供內容產生中文問題的專家。
    你的目標是幫助使用者了解文件內容。
    你可以通過詢問和以下內容有關的問題來做到這一點：

    ------------
    源大環能股份有限公司
誠信經營守則
一、目的
為建立誠信經營之企業文化及健全發展，提供其建立良好商業運作之參考架構，特訂定
本守則。
二、範圍
本公司、子公司及其他具有實質控制能力之機構與組織。
三、權責
董事長室：負責定期檢視內部獨立檢舉信箱及保護檢舉人員。
四、名詞定義
本守則所稱利益，其利益係指任何有價值之事物，包括任何形式或名義之金錢、餽贈、
佣金、職位、服務、優待、回扣等。但屬正常社交禮俗，且係偶發而無影響特定權利義
務之虞時，不在此限。
五、作業說明
1.禁止不誠信行為
本公司之董事、經理人、受僱人、受任人或具有實質控制能力者(以下簡稱本公司人員)，
於從事商業行為之過程中，不得直接或間接提供、承諾、要求或收受任何不正當利益，
或做出其他違反誠信、不法或違背受託義務等不誠信行為，以求獲得或維持利益。(以
下簡稱不誠信行為)
前項行為之對象，包括公職人員、參政候選人、政黨或黨職人員，以及任何公、民營
企業或機構及其董事（理事）、監察人（監事）、經理人、受僱人、實質控制者或其他
利害關係人。
2.法令遵循
    ------------

    產生中文問題，確保不要遺失任何重要信息。

    QUESTIONS:
    


c:\Users\jywun\anaconda3\envs\module\Lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [ ]:
print(questions)


    你是一個根據提供內容產生中文問題的專家。
    你的目標是幫助一個使用者了解文件內容。我們已經收到一些生成出來的問題，但還不夠全面: 
    你是一個根據提供內容產生中文問題的專家。
    你的目標是幫助一個使用者了解文件內容。我們已經收到一些生成出來的問題，但還不夠全面: 
    你是一個根據提供內容產生中文問題的專家。
    你的目標是幫助一個使用者了解文件內容。我們已經收到一些生成出來的問題，但還不夠全面: 
    你是一個根據提供內容產生中文問題的專家。
    你的目標是幫助一個使用者了解文件內容。我們已經收到一些生成出來的問題，但還不夠全面: 
    你是一個根據提供內容產生中文問題的專家。
    你的目標是幫助一個使用者了解文件內容。我們已經收到一些生成出來的問題，但還不夠全面: 
    你是一個根據提供內容產生中文問題的專家。
    你的目標是幫助一個使用者了解文件內容。我們已經收到一些生成出來的問題，但還不夠全面: 
    你是一個根據提供內容產生中文問題的專家。
    你的目標是幫助一個使用者了解文件內容。我們已經收到一些生成出來的問題，但還不夠全面: 
    你是一個根據提供內容產生中文問題的專家。
    你的目標是幫助一個使用者了解文件內容。我們已經收到一些生成出來的問題，但還不夠全面: 
    你是一個根據提供內容產生中文問題的專家。
    你的目標是幫助使用者了解文件內容。
    你可以通過詢問和以下內容有關的問題來做到這一點：

    ------------
    源大環能股份有限公司
誠信經營守則
一、目的
為建立誠信經營之企業文化及健全發展，提供其建立良好商業運作之參考架構，特訂定
本守則。
二、範圍
本公司、子公司及其他具有實質控制能力之機構與組織。
三、權責
董事長室：負責定期檢視內部獨立檢舉信箱及保護檢舉人員。
四、名詞定義
本守則所稱利益，其利益係指任何有價值之事物，包括任何形式或名義之金錢、餽贈、
佣金、職位、服務、優待、回扣等。但屬正常社交禮俗，且係偶發而無影響特定權利義
務之虞時，不在此限。
五、作業說明
1.禁止不誠信行為
本公司之董事、經理人、受僱人、受任人或具有實質控制能力者(以下簡稱本公司人員)，
於從事商業行為之過程中，不得直接或間接提供、承諾、要求或收受任何